In [57]:
import cv2
import numpy as np 

In [58]:
# load Yolo
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []
with open("coco.names", "r")as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [54]:
#loading image
img = cv2.imread("test-image.jpg")
img = cv2.resize(img, None, fx=2.5, fy=2.5)
height, width, channels = img.shape

blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)


net.setInput(blob)
outs = net.forward(output_layers)
print(outs)

[array([[0.03708483, 0.04561232, 0.4647258 , ..., 0.        , 0.        ,
        0.        ],
       [0.04605443, 0.03018425, 0.31607994, ..., 0.        , 0.        ,
        0.        ],
       [0.04627465, 0.03280478, 0.83050907, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.95988727, 0.95052016, 0.42198214, ..., 0.        , 0.        ,
        0.        ],
       [0.96299464, 0.9636949 , 0.30896086, ..., 0.        , 0.        ,
        0.        ],
       [0.96575606, 0.96277374, 0.88075316, ..., 0.        , 0.        ,
        0.        ]], dtype=float32), array([[0.01977936, 0.02624633, 0.04531513, ..., 0.        , 0.        ,
        0.        ],
       [0.02069391, 0.02113351, 0.3559158 , ..., 0.        , 0.        ,
        0.        ],
       [0.02491931, 0.01596277, 0.0789665 , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.9741496 , 0.97720283, 0.04959772, ..., 0.        , 0.        ,
        0.        ],
       [0.98274577

In [55]:
# Showing informations on the screen
class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            
            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)
print(len(boxes))
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
print(indexes)
font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        color = colors[i]
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img, label, (x, y + 30), font, 1, color, 3)

30
[[28]
 [ 3]
 [ 4]
 [16]
 [10]
 [ 8]
 [ 9]]


In [ ]:
cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()